<a href="https://colab.research.google.com/github/adalbertii/Seci-konwolucyjne/blob/main/keras_convolutional_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Educational Friday

---
Developed by Wojciech Michalski

---



# Construction of convolutional neural networks - binary classification case





Using the Keras library

> Convolutional networks learn local patterns. The patterns recognized by the convolutional network are independent of their position in the image.

 > The advantage of CNN is the ability to teach a spatial hierarchy of patterns. The first layers learn small local patterns, such as edges, and subsequent layers will learn larger structures consisting of elements recognized by the initial layers.



### Importing required libraries


In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

### Download training and test data
We will download the data from Amazon S3 storage.

The name of the bucket is: `ml-repository-crackers`.

The zipped file `dogs-vs-cats.zip` is just over 812 MB and is stored in the EU (Ireland) region.

In [ ]:
%%bash
rm -rf /content/*

pwd
wget --output-document=cats_and_dogs.zip --quiet https://ml-repository-krakers.s3-eu-west-1.amazonaws.com/kaggle+/cats_and_dogs/dogs-vs-cats.zip
ls

# Data unpacking

In [ ]:
%%bash
unzip cats_and_dogs.zip
# rm -rf cats_and_dogs.zip

rm -rf sampleSubmission.csv

unzip -q train.zip -d /content/kaggle_original_data
rm -rf train.zip
cp /content/kaggle_original_data/train/* /content/kaggle_original_data/
rm -rf /content/kaggle_original_data/train

unzip -q /content/test1.zip
# rm -rf test1.zip
# rm -rf cats_and_dogs.zip
ls

# Preparation of the appropriate directory structure

The data will be stored in the directory:

> `/content/cats_and_dogs`

We will divide this directory into three subdirectories:
 * train
 * valid
 * test

In the training set, we will put 2,000 samples (1,000 photos of dogs and 1,000 photos of cats), 1,000 samples (500 photos of dogs and 500 photos of cats) will go to the validation set, and the last 1,000 samples (500 photos of dogs and 500 photos of cats) will go to the test set.

In [ ]:

original_dataset_dir = '/content/kaggle_original_data'

base_dir = '/content/cats_and_dogs'

if not os.path.exists(base_dir):
    os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

valid_cats_dir = os.path.join(valid_dir, 'cats')
valid_dogs_dir = os.path.join(valid_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

dirs = [train_cats_dir, train_dogs_dir, valid_cats_dir, valid_dogs_dir, test_cats_dir, test_dogs_dir]

for directory in dirs:
    if not os.path.exists(directory):
        os.mkdir(directory)


# separate the fhotos
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(valid_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(valid_dogs_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


Checking the correctness of the division


In [ ]:
print('Number of cats - training set', len(os.listdir(train_cats_dir)))
print('Number of cats - validation set', len(os.listdir(valid_cats_dir)))
print('Number of cats - test set', len(os.listdir(test_cats_dir)))

print('Number of dogs - training set', len(os.listdir(train_dogs_dir)))
print('Number of dogs - validation set', len(os.listdir(valid_dogs_dir)))
print('Number of dogs - test set', len(os.listdir(test_dogs_dir)))


### Look at the sample pictures - cats and dogs
> Tip: Use the slider on the right to select a different image index.

In [ ]:
import PIL
import PIL.Image

In [ ]:
index = 438 #@param {type:'slider', min:0, max:999}
img_path = '/content/cats_and_dogs/train/cats/cat.' + str(index) +'.jpg'
PIL.Image.open(img_path)

In [ ]:
index = 438 #@param {type:'slider', min:0, max:999}
img_path = '/content/cats_and_dogs/train/dogs/dog.' + str(index) +'.jpg'
PIL.Image.open(img_path)

### Helpful function definition

In [ ]:
def make_accuracy_plot(history):
    """
    The function returns the accuracy plot of the model on the training set
    and validation.
    """
    acc, val_acc = history.history['accuracy'], history.history['val_accuracy']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(10, 8))
    plt.plot(epochs, acc, label='Accuracy of training', marker='o')
    plt.plot(epochs, val_acc, label='Validation accuracy', marker='o')
    plt.legend()
    plt.title('Accuracy of training and validation')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()

def make_loss_plot(history):
    """
    The function returns the loss plot of the model on the training set
    and validation.
    """
    loss, val_loss = history.history['loss'], history.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(10, 8))
    plt.plot(epochs, loss, label='Training loss', marker='o')
    plt.plot(epochs, val_loss, label='Validation loss', marker='o')
    plt.legend()
    plt.title('Loss of training and validation')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()


### Building the model


> The Conv2D layer is a convolution layer in which at least three parameters must be specified:
- the number of filters that will extract the features,
- the size of the kernel (the size of the convolution window, usually 3x3 or 5x5) and
- the activation function (the most commonly used `relu` function).

In addition, in the first layer, you must specify the `input_shape` parameter, which takes the size of the input data. In our case `(150, 150, 3)`. The first two values ​​determine the width and height of the image, while the third determines the color depth, in this case 3.

> The MaxPooling2D layer is a scaling operation. In simple terms, it consists in reducing dimensionality by passing through the image with extraction windows that return the maximum observed value in a given window (usually a 2x2 window size, with a shift step of 2), thus helping to reduce the size of the input data to the next layer by half, which significantly speeds up the process learning.

> The Flatten layer flattens our data in order to combine them with dense layers at the end of building the model. The last activation function will be the `sigmoid` function, which will return the probabilities of the image belonging to a particular class.

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))
model.summary()


### Model compilation



In [ ]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
             loss='binary_crossentropy',
             metrics=['accuracy'])

### Step 5 - Processing the data into the model

Since our data is non-standardized (different sizes of images), we need to process it so that it is presented in the form of tensors of floating point values ​​(float). Our data is currently in JPG files. In the data processing process, the `ImageDataGenerator` class built into `Keras` will help us. All images from the training and validation sets will be scaled to 150x150 pixels.

#### In short, we will do:
- upload files in JPG format
- decode JPG format to pixel grid in RGB format
- save the data in the tensor format of floating point numbers
- scale the pixel values ​​to the range [0, 1] (neural networks are better at dealing with small input values)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# we scale all images by a factor of 1/255
train_datagen = ImageDataGenerator(rescale=1./255.)
valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=20,
                                                   class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                   target_size=(150, 150),
                                                   batch_size=20,
                                                   class_mode='binary')

### Checking the correct shape of the training data

In [ ]:
for data_batch, labels_batch in train_generator:
    print('Batch data shape:', data_batch.shape)
    print('Batch data label shape:', labels_batch.shape)
    break

### Model training

We will save the trained model to the file `cats_and_dogs_small_1.h5`.



In [ ]:
import time

tic = time.time()
history = model.fit_generator(generator=train_generator,
                             steps_per_epoch=100,
                             epochs=20,
                             validation_data=valid_generator,
                             validation_steps=50)

toc = time.time()
print('Processing time: {}'.format(toc - tic))
model.save('wmi_small_model_1.h5')

### Training and validation accuracy graph

In [ ]:
make_accuracy_plot(history)

### Training and validation loss graph

In [ ]:
make_loss_plot(history)

### Data Augmentation

Particularly useful in cases of insufficient training data. It consists in various transformations of the input data through operations such as cropping, rotation or zooming in order to generate new input data.




### Building a model using images generated using data augmentation

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.summary()

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# we do not modify the validation data!!!
valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=20,
                                                   class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                   target_size=(150, 150),
                                                   batch_size=20,
                                                   class_mode='binary')


### Model training

In [ ]:
history = model.fit_generator(generator=train_generator,
                             steps_per_epoch=100,
                             epochs=20,    # 100
                             validation_data=valid_generator,
                             validation_steps=50)

### Save model to file

In [ ]:
model.save('wmi_small_model_2.h5')

### Training and validation accuracy graph

In [ ]:
make_accuracy_plot(history)

### Training and validation loss graph

In [ ]:
make_loss_plot(history)

### Transfer Learning -

is based on the use of an already overtrained model, usually on a very large input set,
We will use the architecture of the VGG16 model (containing 16 layers) trained on the `Imagenet` set.
The Imagenet collection consists of 1.4 million images divided into 1000 classes, which also include different breeds of dogs and cats.
The finished model is available in the Keras library.

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,    # whether to join the upper part of the network
                 input_shape=(150, 150, 3))

conv_base.summary()

## Feature extraction with data augmentation

We will use the advantages of Transfer Learning as well as data augmentation techniques. We will add the last three layers to the downloaded model: a Flatten layer and two dense layers (Dense).

In [ ]:
from keras import models
from keras import layers


model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.summary()

### Train the model from start to finish with a frozen convolutional base

We will now freeze the weights of the VGG16 network to prevent these weights from being updated in the training process. If we did not, the features that the model learned could be modified during training

In [ ]:
conv_base.trainable = False

### Data augmentation stage

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers


train_datagen = ImageDataGenerator(rescale=1./255.,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

# validation data cannot be modified!!!
test_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=20,
                                                   class_mode='binary')
valid_generator = test_datagen.flow_from_directory(valid_dir,
                                                   target_size=(150, 150),
                                                   batch_size=20,
                                                   class_mode='binary')

### Compilation and training of an extended model (excluding convolution layers)



In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss='binary_crossentropy',
             metrics=['accuracy'])

history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=50,
                             validation_data=valid_generator,
                             validation_steps=50,
                             verbose=2)

### Accuracy Level Verification

In [ ]:
make_accuracy_plot(history)

### Loss function distribution

In [ ]:
make_loss_plot(history)

### Tuning



This technique consists in 'unfreezing' several upper layers of the frozen base of the model. We will tune the last three convolutional layers (`block5_conv1, block5_conv2, block5_conv3`).



In [ ]:
conv_base = VGG16(weights='imagenet',
                 include_top=False,    # whether to include the upper part of the network
                 input_shape=(150, 150, 3))

conv_base.summary()

In [ ]:
conv_base.trainable = True

def print_layers(model):
    for layer in model.layers:
        print('layer_name: {:12}, trainable: {}'.format(layer.name, layer.trainable))

print_layers(conv_base)

In [ ]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

print_layers(conv_base)

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()


### Model training

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=50,
                             validation_data=valid_generator,
                             validation_steps=50)

In [ ]:
make_accuracy_plot(history)

In [ ]:
make_loss_plot(history)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(150, 150),
                                                 batch_size=20,
                                                 class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('Testing Accuracy:', test_acc)

##Conclusions

* Convolutional neural networks are the best machine learning models for image processing tasks
* Convolutional networks can be trained even on small data sets, for this purpose data augmentation and transfer learning are helpful (example: medical photos)
* The possibility of using transfer learning allows you to train models in a much shorter time with much greater efficiency of such a model compared to a model based only on the data we have
* Tuning techniques allow you to adapt previously learned models to our problem, which should in fact lead to improved model performance
* As it is commonly believed about neural networks as "black boxes", in the case of convolutional networks, the individual stages of the network's operation are easy to visualize.

